# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 3 - Flux Analysis & Constraint-based modeling

## Working with a GEM model

COBRApy can be used to read a GEM model in an SBML format file.
For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: ../data/e_coli_core.xml

You can read more about _E. coli_ core model (Orth et al., 2010) in the following links:
- https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1
- http://bigg.ucsd.edu/models/e_coli_core

The reactions, metabolites, and genes encoded into an SBML format file can be parsed by COBRApy. These contents are loaded into Python objects simple to use, namely `cobra.Reaction`, `cobra.Metabolite`, and `cobra.Gene`

The model itself will be available as an `cobra.Model` object containing all these attributes.

In [1]:
# importing cobra
import cobra

In [2]:
# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

Name,e_coli_core
Memory address,18f50695608
Number of metabolites,72
Number of reactions,95
Number of genes,137
Number of groups,0
Objective expression,1.0*BIOMASS_Ecoli_core_w_GAM - 1.0*BIOMASS_Ecoli_core_w_GAM_reverse_712e5
Compartments,"extracellular space, cytosol"


In [3]:
# retrieving the first five reactions of the model
model.reactions[0:5]

[<Reaction PFK at 0x18f505b5ac8>,
 <Reaction PFL at 0x18f505b5a88>,
 <Reaction PGI at 0x18f505bce48>,
 <Reaction PGK at 0x18f505cad48>,
 <Reaction PGL at 0x18f505cadc8>]

In [4]:
# inspecting the first reaction of the model
model.reactions[0]

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x18f505b5ac8
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [5]:
# inspecting a reaction by its ID
model.reactions.get_by_id("PFK")

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x18f505b5ac8
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [6]:
# inspecting a reaction using the ID as attribute
model.reactions.PFK

Reaction identifier,PFK
Name,Phosphofructokinase
Memory address,0x18f505b5ac8
Stoichiometry,"atp_c + f6p_c --> adp_c + fdp_c + h_c ATP C10H12N5O13P3 + D-Fructose 6-phosphate --> ADP C10H12N5O10P2 + D-Fructose 1,6-bisphosphate + H+"
GPR,b3916 or b1723
Lower bound,0.0
Upper bound,1000.0


In [7]:
# retrieving the first five metabolites of the model
model.metabolites[0:5]

[<Metabolite glc__D_e at 0x18f50604088>,
 <Metabolite gln__L_c at 0x18f50604048>,
 <Metabolite gln__L_e at 0x18f50604188>,
 <Metabolite glu__L_c at 0x18f506048c8>,
 <Metabolite glu__L_e at 0x18f50608448>]

In [8]:
# inspecting the first metabolite of the model
model.metabolites[0]

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x18f50604088
Formula,C6H12O6
Compartment,e
In 2 reaction(s),"GLCpts, EX_glc__D_e"


In [9]:
# inspecting a metabolite by its ID
model.metabolites.get_by_id("glc__D_e")

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x18f50604088
Formula,C6H12O6
Compartment,e
In 2 reaction(s),"GLCpts, EX_glc__D_e"


In [10]:
# inspecting a metabolite using the ID as attribute
model.metabolites.glc__D_e

Metabolite identifier,glc__D_e
Name,D-Glucose
Memory address,0x18f50604088
Formula,C6H12O6
Compartment,e
In 2 reaction(s),"GLCpts, EX_glc__D_e"


In [11]:
# retrieving the first five genes of the model
model.genes[0:5]

[<Gene b1241 at 0x18f5064cdc8>,
 <Gene b0351 at 0x18f5064cbc8>,
 <Gene s0001 at 0x18f506561c8>,
 <Gene b1849 at 0x18f50656988>,
 <Gene b3115 at 0x18f50656fc8>]

In [12]:
#inspecting the first gene of the model
model.genes[0]

Gene identifier,b1241
Name,adhE
Memory address,0x18f5064cdc8
Functional,True
In 2 reaction(s),"ALCD2x, ACALD"


In [13]:
# inspecting a gene by its ID
model.genes.get_by_id('b1241')

Gene identifier,b1241
Name,adhE
Memory address,0x18f5064cdc8
Functional,True
In 2 reaction(s),"ALCD2x, ACALD"


In [14]:
# inspecting a gene using the ID as attribute
model.genes.b1241

Gene identifier,b1241
Name,adhE
Memory address,0x18f5064cdc8
Functional,True
In 2 reaction(s),"ALCD2x, ACALD"


### Reactions

In COBRApy, reactions are objects that can be inspected. A `Reaction` object usually contains the following information:
- name;
- metabolites;
- stoichiometry;
- genes;
- formula;
- reversibility;
- bounds;
- gpr;

In [15]:
#inspecting the reaction name, formula, metabolites, and stoichiometry.
reaction = model.reactions.get_by_id('ACALD')

print('Reaction name:', reaction.name, '\n')
print('Reaction formula', reaction.reaction, '\n')
for metabolite, coefficient in reaction.metabolites.items():
    print(metabolite, '->', coefficient)

Reaction name: Acetaldehyde dehydrogenase (acetylating) 

Reaction formula acald_c + coa_c + nad_c <=> accoa_c + h_c + nadh_c 

acald_c -> -1.0
coa_c -> -1.0
nad_c -> -1.0
accoa_c -> 1.0
h_c -> 1.0
nadh_c -> 1.0


In [16]:
# inspecting reversibility and flux bounds.
print(reaction.lower_bound, "< ACALD <", reaction.upper_bound, '\n')
print('Reaction reversibility:', reaction.reversibility, '\n')
print('Reaction bounds:', reaction.bounds)

-1000.0 < ACALD < 1000.0 

Reaction reversibility: True 

Reaction bounds: (-1000.0, 1000.0)


In [17]:
# change reaction bounds
reaction.bounds = (0, 1000)
print(reaction.lower_bound, "< ACALD <", reaction.upper_bound, '\n')
print('Reaction reversibility:', reaction.reversibility, '\n')

0 < ACALD < 1000 

Reaction reversibility: False 



### Metabolites

In COBRApy, metabolites are objects that can be inspected. A `Metabolite` object usually contains the following information:
- name;
- chemical formula;
- compartment;
- reactions;

In [18]:
# inspecting metabolite name, chemical formula, compartment and reactions.
metabolite = model.metabolites.get_by_id('glc__D_e')

print('Metabolite name:', metabolite.name, '\n')
print('Metabolite chemical formula:', metabolite.formula, '\n')
print('Metabolite compartment:', metabolite.compartment, '\n')
for _reaction in metabolite.reactions:
    print(_reaction.id, ':', _reaction.reaction)

Metabolite name: D-Glucose 

Metabolite chemical formula: C6H12O6 

Metabolite compartment: e 

GLCpts : glc__D_e + pep_c --> g6p_c + pyr_c
EX_glc__D_e : glc__D_e <=> 


### Genes

In COBRApy, genes are objects that can be inspected. A `Gene` object usually contains the following information:
- name;
- reactions;

In [19]:
# inspect gene name and reactions.
gene = model.genes.get_by_id('b0351')

print('Gene name:', gene.name, '\n')
for reaction_ in gene.reactions:
    print(reaction_.id, ':', reaction.gene_reaction_rule)

Gene name: mhpF 

ACALD : b0351 or b1241


### Exchanges

In COBRApy, exchanges are reaction objects that can be inspected in the model. These special reactions define the environmental conditions (e.g. medium) of the model. According to the lower and upper bound of each exchange reaction, one can visualize the model's uptake and secretion metabolites.

In [20]:
# inspecting the exchange reactions.
for exchange in model.exchanges:
    print(exchange.name, '->', exchange.bounds)

Acetate exchange -> (0.0, 1000.0)
Acetaldehyde exchange -> (0.0, 1000.0)
2-Oxoglutarate exchange -> (0.0, 1000.0)
CO2 exchange -> (-1000.0, 1000.0)
Ethanol exchange -> (0.0, 1000.0)
Formate exchange -> (0.0, 1000.0)
D-Fructose exchange -> (0.0, 1000.0)
Fumarate exchange -> (0.0, 1000.0)
D-Glucose exchange -> (-10.0, 1000.0)
L-Glutamine exchange -> (0.0, 1000.0)
L-Glutamate exchange -> (0.0, 1000.0)
H+ exchange -> (-1000.0, 1000.0)
H2O exchange -> (-1000.0, 1000.0)
D-lactate exchange -> (0.0, 1000.0)
L-Malate exchange -> (0.0, 1000.0)
Ammonia exchange -> (-1000.0, 1000.0)
O2 exchange -> (-1000.0, 1000.0)
Phosphate exchange -> (-1000.0, 1000.0)
Pyruvate exchange -> (0.0, 1000.0)
Succinate exchange -> (0.0, 1000.0)
